In [ ]:
!pip install transformers datasets evaluate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer

base_model = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
guanco_dataset = "mlabonne/guanaco-llama2-1k"
new_model = "llama-1.1B-chat-guanaco"

dataset = load_dataset(guanco_dataset, split="train")
model = AutoModelForCausalLM.from_pretrained(base_model, device_map='auto')
model.config.use_cache= False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'


In [ ]:
#run inference
logging.set_verbosity(logging.CRITICAL)
prompt = "Who is Napoleon Bonaparte?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
# print(result[0]['generated_text'])
print(result)

[{'generated_text': '<s>[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST] Who is Napoleon Bonaparte? [/INST]\n[INST]'}]


In [ ]:
import os
os.environ["WANDB_MODE"]= "disabled"

In [ ]:
import torch

peft_params = LoraConfig(lora_alpha=16, #multiplier of lora output when its added to the full forward output
                         lora_dropout=0.1,
                         r=64,
                         bias="none",
                         task_type="CASUAL_LM")

training_params = TrainingArguments(output_dir='./results',
                                    num_train_epochs=2,
                                    per_device_train_batch_size=2,
                                    gradient_accumulation_steps=16,
                                    optim="adamw_torch",
                                    save_steps=25,
                                    logging_steps=1,
                                    learning_rate=2e-4,
                                    weight_decay=0.001,
                                    fp16=True,
                                    bf16=False,
                                    max_grad_norm=0.3,
                                    max_steps=-1,
                                    warmup_ratio=0.03,
                                    group_by_length=True,
                                    lr_scheduler_type="cosine")

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field = "text",
    max_seq_length= None,
    tokenizer= tokenizer,
    args = training_params,
    packing=False
)

import gc
gc.collect()
torch.cuda.empty_cache()

trainer.train()
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


{'loss': 1.4743, 'grad_norm': 0.0625004842877388, 'learning_rate': 0.0001, 'epoch': 0.032}
{'loss': 1.6063, 'grad_norm': 0.07649805396795273, 'learning_rate': 0.0002, 'epoch': 0.064}
{'loss': 1.7181, 'grad_norm': 0.0671316534280777, 'learning_rate': 0.0001998629534754574, 'epoch': 0.096}
{'loss': 1.7199, 'grad_norm': 0.0916563868522644, 'learning_rate': 0.00019945218953682734, 'epoch': 0.128}
{'loss': 1.9804, 'grad_norm': 0.10270264744758606, 'learning_rate': 0.00019876883405951377, 'epoch': 0.16}
{'loss': 2.1968, 'grad_norm': 0.14221307635307312, 'learning_rate': 0.00019781476007338058, 'epoch': 0.192}
{'loss': 2.7359, 'grad_norm': 0.27412551641464233, 'learning_rate': 0.00019659258262890683, 'epoch': 0.224}
{'loss': 1.6217, 'grad_norm': 0.08943010121583939, 'learning_rate': 0.00019510565162951537, 'epoch': 0.256}
{'loss': 1.6594, 'grad_norm': 0.10465283691883087, 'learning_rate': 0.00019335804264972018, 'epoch': 0.288}
{'loss': 1.765, 'grad_norm': 0.09990952908992767, 'learning_rate'

('llama-1.1B-chat-guanaco/tokenizer_config.json',
 'llama-1.1B-chat-guanaco/special_tokens_map.json',
 'llama-1.1B-chat-guanaco/tokenizer.model',
 'llama-1.1B-chat-guanaco/added_tokens.json',
 'llama-1.1B-chat-guanaco/tokenizer.json')

In [ ]:
prompt = "Who is Napoleon Bonaparte?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe("<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Napoleon Bonaparte? [/INST] Napoleon Bonaparte was a French military leader who led the French Revolution and the Napoleonic Empire. He was also a politician, philosopher, and writer. He was born in 1769 and died in 1821.
Napoleon Bonaparte was a French military leader who led the French Revolution and the Napoleonic Empire. He was also a politician, philosopher, and writer. He was born in 1769 and died in 1821. Napoleon Bonaparte was a French military leader who led the French Revolution and the Napoleonic Empire. He was also a politician, philosopher, and writer. He was born in 1769 and died in 1821.
Napoleon Bonaparte was a French military leader who led the French Revolution and the Napoleonic Empire. He was also


In [ ]:
prompt = "Who is Donal trump?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Donal trump? [/INST] Donal trump is a famous American politician who is currently serving as the 45th President of the United States of America. He was born on 14th September 1946 in New York City, New York, USA. He is the son of Donald Trump and Ivana Trump. He has two siblings, Donald Jr. and Ivanka. He graduated from the University of Pennsylvania with a degree in economics. He has been married to Melania Trump since 2005 and they have three children, Barron, Ivanka, and Eric. He is a Republican and has been a member of the Trump administration since 2017. He is known for his business acumen and his ability to negotiate deals. He is also known for his controversial statements and his use of social media. He is a popular figure in the political world
